In [ ]:
#!pip install openai
#!pip install langchain-core==0.2.17
#!pip install langchain-openai==0.1.17
#!pip install pydantic==1.10.8
#!pip install pydantic_core==2.18.2
!pip install -U openai langchain langchain-openai

In [1]:
# from langchain_core.pydantic_v1 import BaseModel, Field # 예전 방식

# from pydantic import BaseModel # 새로운 방식

from pydantic.v1 import BaseModel # pydantic.v1 호환이 필요한 경우

In [2]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.chat_models import ChatOpenAI

prompt = ChatPromptTemplate.from_template("{topic}에 대한 간단한 농담을 말해줘.")
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser # LCEL의 기본 파이프라인

chain.invoke({"topic": "아이스크림"})

/tmp/ipykernel_36680/3759826758.py:6: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  model = ChatOpenAI(model="gpt-4o-mini")


'왜 아이스크림은 항상 행복할까요?\n\n왜냐하면 항상 "스쿱"을 받으니까요! 🍦😄'

In [3]:
!pip install docarray tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [4]:
# pip install langchain docarray tiktoken

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough
from langchain_community.chat_models import ChatOpenAI
from langchain_community.embeddings import OpenAIEmbeddings

vectorstore = DocArrayInMemorySearch.from_texts(
    ["해리슨은 학교에서 일합니다.", "곰은 꿀을 좋아해"],
    embedding=OpenAIEmbeddings(),
)
retriever = vectorstore.as_retriever()

template = """다음 지문에만 근거해서 질문에 답하세요:
{context}

질문: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()
output_parser = StrOutputParser()

setup_and_retrieval = RunnableParallel( # 여러개의 Runnable을 병렬적으로 실행
    {"context": retriever, "question": RunnablePassthrough()} # RunnablePassthrough는 값을 입력받아 그대로 전달하는 객체
)
chain = setup_and_retrieval | prompt | model | output_parser

# retrieval | prompt | model | output. RAG의 기본 파이프라인

/tmp/ipykernel_36680/755495277.py:12: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding=OpenAIEmbeddings(),
/home/ubuntu/.local/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
chain.invoke("해리슨은 어디에서 일하나요?")

'답변: 해리슨은 학교에서 일합니다.'

In [6]:

chain.invoke("곰은 무엇을 좋아하나요?")

'답변: 곰은 꿀을 좋아합니다.'

In [8]:
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

prompt = ChatPromptTemplate.from_template(
    "{topic}에 대한 간단한 농담을 해줘."
)
output_parser = StrOutputParser()
model = ChatOpenAI(model="gpt-4o-mini")
chain = (
    {"topic": RunnablePassthrough()} 
    | prompt
    | model
    | output_parser
)

for chunk in chain.stream("ice cream"):
    print(chunk, end="", flush=True)

왜 아이스크림은 항상 행복할까요?

왜냐하면 항상 "스쿱"을 즐기니까요! 🍦😄

In [9]:
chain.batch(["아이스크림", "스파게티", "만두"])

# stream: 입력에 대해 청크를 스트리밍
# invoke: 입력에 대해 체인을 호출
# batch: 입력목록에 대해 체인을 배치로 호출

['왜 아이스크림은 항상 기분이 좋을까요? \n\n왜냐하면 항상 "스쿱"이 좋거든요! 🍦😄',
 '스파게티가 파스타 모임에서 왜 항상 웃고 있을까요? \n\n왜냐하면 항상 "면"세가 좋으니까요! 🍝😄',
 '왜 만두가 항상 행복할까요? \n\n늘 속이 꽉 차 있으니까요! 😄']

In [10]:
await chain.ainvoke("ice cream")

# astream: 비동기로 입력에 대해 청크를 스트리밍
# ainvoke: 비동기로 입력에 대해 체인을 호출
# abatch: 비동기로 입력목록에 대해 체인을 배치로 호출

'왜 아이스크림은 항상 행복할까요? \n\n왜냐하면 항상 "스쿱"을 가지고 있으니까요! 🍦😄'

In [11]:
await chain.abatch(
    ["아이스크림", "스파게티", "만두"]
)

['아이스크림이 왜 학교에 가고 싶어 했을까요? \n\n왜냐하면 더 많은 "스쿨"을 갖고 싶었거든요! 🍦😄',
 '왜 스파게티는 항상 좋은 기분을 유지할까요? \n\n왜냐하면 항상 "파스타"한 기분이니까요! 🍝😄',
 '왜 만두는 항상 행복할까요? \n\n왜냐하면 속이 꽉 차있어서 항상 기분이 좋거든요! 😄']

In [12]:
chain.input_schema.model_json_schema()

{'properties': {'root': {'title': 'Root'}},
 'required': ['root'],
 'title': 'RunnableParallel<topic>Input',
 'type': 'object'}

In [13]:
chain.output_schema.model_json_schema()

{'title': 'StrOutputParserOutput', 'type': 'string'}